In [1]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
db.reset()

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
wallet.import_key('test', '5KFQ5opDDCADMZwwwsKF89przC9pZc9Dc4Ed2GYbGo8S32Yy2Gw')
wallet.import_key('test', '5KQwrPbwdL6PhXujxW37FSSQZ1JiwsST4cqQzDeyXtP79zkvFD3')


def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True
#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])


In [12]:
eosapi.get_public_key('5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

'EOS7u2gnQzoybz6YZvoi5RY6rMqkbAycDBFEFGhsvHEDmB7tffc2h'

In [37]:
code = '''
import db
def apply(receiver, code, action):
    if not receiver == code:
        return

    code = receiver
    scope = 'scope'
    table = 'table'
    payer = receiver
    id = 'hello'
    data = read_action_data()
#    print(data)
    
    itr = db.find_i64(code, scope, table, id)
#    print(itr)
    if N('setvalue') == action:
        if itr < 0:
            db.store_i64(scope, table, payer, id, b'world')
        else:
            old_data = db.get_i64(itr)
            print(old_data)
            db.update_i64(itr, receiver, data)
    elif N('getvalue') == action:
        data = db.get_i64(itr)
        print(data)
'''

abi = ''
account_name = 'helloworld11'
publish_contract(account_name, code, abi)
try:
    r = eosapi.push_action(account_name, 'setvalue', b'hello,world', {account_name:'active'})
    print(r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

b'hello,world'

273


In [31]:
eosapi.call_contract(name, 'setvalue', b'hello'.hex())

{
    "results": {
        "error": {
            "name": "table_access_violation",
            "code": 3160002,
            "what": "Table access violation (3160002)\ncan not write to read only database\npending console output: b'hello,world'\n\n"
        }
    }
}

In [32]:
eosapi.call_contract(name, 'getvalue', b'hello'.hex())

{
    "results": {
        "output": "b'hello,world'\n"
    }
}